In [1]:
import ee
import geemap
import shutil
import os
import sys
import time
sys.path.insert(0, '../src/')
import preproc

In [2]:
ee.Authenticate()
ee.Initialize()
coordenadas = "-51.881964,3.928400,-51.746512,3.791524"
x1, y1, x2, y2 = coordenadas.split(",")

datas = "2018-01-01,2020-12-31"
inicio, fim = datas.split(",")
escala = 30
dummy_value = 99999

geom = ee.Geometry.Polygon([[[float(x1),float(y2)],
                             [float(x2),float(y2)],
                             [float(x2),float(y1)],
                             [float(x1),float(y1)],
                             [float(x1),float(y2)]]])


sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD')\
    .filterBounds(geom)\
    .filterDate(inicio,fim)\
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))\
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))\
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    
v_emit_asc = sentinel1.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
v_emit_desc = sentinel1.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))

image = ee.Image(dummy_value).blend(v_emit_desc.map(preproc.ee_add_amplitude).select('amplitude').toBands())
image_names = image.bandNames().getInfo()

In [3]:
latitude_central = (float(x1)+float(x2))/2
longitude_central = (float(y1)+float(y2))/2

my_map = geemap.Map(center=(longitude_central, latitude_central), zoom=13)
my_map.addLayer(geom)

rgb = ee.Image.rgb(
    v_emit_desc.mean().select('VV'),
    v_emit_desc.mean().select('VH'),
    v_emit_desc.mean().select('VV').divide(v_emit_desc.mean().select('VH'))
).clip(geom)

my_map.addLayer(rgb, {'min': [-25,-25,0], 'max': [0,0,2]})

my_map

Map(center=[3.859962, -51.814238], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sear…

In [4]:
image_length = len(image.bandNames().getInfo())
print(image_length, 'images found.')

81 images found.


In [5]:
start_task = time.time()
for i in list(range(image_length)):
    file_name = 'raw_' + str(i) + '.tif'
    geemap.ee_export_image(
        image.select(i), 
        file_name, 
        scale=escala, 
        crs='EPSG:4326', 
        region=geom)
end_task = time.time()
print("Images extracted in " + str(round(end_task - start_task, 2)) + " seconds")

Generating URL ...
Please wait ...
Data downloaded to c:\Users\jpmel\Desktop\Projects\specs\data\raw_0.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\jpmel\Desktop\Projects\specs\data\raw_1.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\jpmel\Desktop\Projects\specs\data\raw_2.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\jpmel\Desktop\Projects\specs\data\raw_3.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\jpmel\Desktop\Projects\specs\data\raw_4.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\jpmel\Desktop\Projects\specs\data\raw_5.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\jpmel\Desktop\Projects\specs\data\raw_6.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\jpmel\Desktop\Projects\specs\data\raw_7.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\jpmel\Desktop\Projects\specs\data\raw_8.tif
Generating URL ...
Please wait ...
Da

In [6]:
files = [f for f in os.listdir() if f.startswith('raw_')]
for file in files:
    shutil.move(file,'raw/'+ file)